## Team name: Group4
### Simone Bellini, Student ID 567572, Kaggle name: Simone Bellini
### Miguel Baptista, Student ID 566511 Kaggle name: Miguel Baptista
### Laura Rodriguez, Student ID 566591 Kaggle name: lrodriguez2

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder
from datetime import date

from matplotlib import rcParams
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import lightgbm as lgbm
import warnings

%load_ext autoreload
%matplotlib inline

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


warnings.filterwarnings("ignore")

-Table description:
--
-beer_train.csv - the training set
-beer_test.csv - the test set
-sampleSubmission.csv - a sample submission file in the correct format
-id_store_sku.csv - id - store_id - sku_name mapping
-sku_features.csv - SKU Features

-Feature description:
--
-Date - calendar date
-ts_id - time-series identification, each of them describe sales of one SKU at one Store
-isPromo - how many types of promotion were active on this day. 0 = no promotions, 1 = a single promotion type (for example, -price discount), 2 = two promotion types simultaneously (for example, both price discount and special placement)-
-Sales - actual sales, target
-id - identifier of rows in the submission file (the same order as in the test set). This is the key column Kaggle uses to match the test labels and your predictions. Order matters
-Store - identifier of the stores (supermarket)
-SKU - Stock Keeping Unit. Identifier of the product
-Segment - price segment of the product
-Pack - package type
-Product - type of the drink
-Brand - product's brand
-Volume - package size

In [ ]:
beer_train = pd.read_csv("data/beer_train.csv")


In [ ]:
beer_test = pd.read_csv("data/beer_test.csv")

In [ ]:
sampleSubmission = pd.read_csv("data/sampleSubmission.csv")

In [ ]:
id_store_sku = pd.read_csv("data/id_store_sku.csv")
id_store_sku.dtypes

In [ ]:
sku_features = pd.read_csv("data/sku_features.csv")

In [ ]:
result = pd.merge(id_store_sku, sku_features, on="SKU")

In [ ]:
final_result = pd.merge(beer_train, result, on="ts_id")
final_result

In [ ]:
final_result_test = pd.merge(beer_test, result, on="ts_id")
final_result_test

# Visualization (EDA)

There are 267 stores and 18 SKU in id_store_sku
There are 19 SKU in sku_features
There are 256 stores, 18 SKU, in the final join

That means that we don't have sales for 11 stores, and a beer is not sold anywhere


There are 1944 ts_id in id_store_sku
There are 1891 ts_id in beer_train and beer_test
We have 173 missing ts_id in beer_train and beer_test
We have 120 missing ts_id in id_store_sku

That means that for 53 ts_ids we don't have sales, for 120 we don't have sales either correspondance with beer/store


Date
For the train, we have 731 data points(days) for each ts_id
For the test, we have 102 data points(days) for each ts_id

In [ ]:
fr = final_result.copy(deep = True)
fr['Date'] = pd.to_datetime(final_result['Date'])

fr = fr[fr['Brand']=="Don"]
fr
res = fr.groupby(fr['Date'].dt.date).mean()  
res
plot_cols = ['Sales','isPromo']#'isPromo'
plot_features = res[plot_cols]
_ = plot_features.plot(subplots=False)

In [ ]:
#Plot
ts_id0 = final_result[final_result['ts_id']==0]
date_time = pd.to_datetime(ts_id0.pop('Date'), format='%Y-%m-%d')
plot_cols = ['Sales','isPromo','Segment']
plot_features = ts_id0[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

### Effectiveness of is_Promo

PACK Can, PET, Bottle
Can : Seasonality around mid-september/begin june 2x (down) and new year/may (up)
PET : september(down)
Bottle : new year (up)

SEGMENT Premium, MainStream, High Margin
Premium : Seasonality new year (up), september/june (down)
MainStream: maybe little trend up, september/june (down) HIGH CORR WITH ISPROMO !!
High Margin: only 51 ts_id, 0 since OCT 2015, HIGH CORR WITH ISPROMO !!

PRODUCT Lager, Energy-drink, Non-alco, Other
Lager: september/june (down) new year (up)
Energy-drink: 0 since OCT 2015, GOOD CORR WITH ISPROMO !!
Non-alco: REAALLY VARIABLE,  new year (up), up in summer IS PROMO USELESS !!
Other: VERY HIGH CORR WITH ISPROMO !!

VOLUME 0.45, 0.5, 1.5
0.45: september/june (down), new year (up)
0.5: september/june (down), new year (up)
1.5: GOOD CORR WITH ISPROMO !!

STORE: TOO MANY

BRAND Carlsberg Flash Tuborg Zatecky Gus Baltika №3 Baltika №7 Baltika №9 Baltika Cooler Baltika Draught Don Firmennoe Priazovskoye
Carlsberg : HIGH CORR WITH ISPROMO !!
Flash : GOOD CORR WITH ISPROMO !!
Holsten : no data
Tuborg : GOOD CORR WITH ISPROMO !!
Zatecky Gus : GOOD CORR WITH ISPROMO !!
Baltika №3 : GOOD CORR WITH ISPROMO !!
Baltika №7 : GOOD CORR WITH ISPROMO !!
Baltika №9 : september/june (down)
Baltika Cooler : HIGH CORR WITH ISPROMO !!
Baltika Draught : HIGH CORR WITH ISPROMO !!
Don : GOOD CORR WITH ISPROMO !!
Firmennoe : GOOD CORR WITH ISPROMO !!
Priazovskoye : random

In [ ]:
#TRAIN
final_result ['ProductPromoEffectiveness'] = np.where(final_result['Product'] == ("Other" or "Energy-drink"), 1, 0)
final_result ['VolumePromoEffectiveness'] = np.where(final_result['Volume'] == 1.5, 1, 0)
final_result ['BrandPromoEffectiveness'] = np.where(final_result['Brand'] != ("Baltika №9" or "Priazovskoye"), 1, 0)

#TEST
final_result_test ['ProductPromoEffectiveness'] = np.where(final_result_test['Product'] == ("Other" or "Energy-drink"), 1, 0)
final_result_test ['VolumePromoEffectiveness'] = np.where(final_result_test['Volume'] == 1.5, 1, 0)
final_result_test ['BrandPromoEffectiveness'] = np.where(final_result_test['Brand'] != ("Baltika №9" or "Priazovskoye"), 1, 0)

# Feature Engineering

### Transform into regression problem

In [ ]:
#TRAIN
final_result['Year'] = pd.to_datetime(final_result['Date']).dt.year
final_result['Month'] = pd.to_datetime(final_result['Date']).dt.month
final_result['Day'] = pd.to_datetime(final_result['Date']).dt.day
final_result['Dayofweek'] = pd.to_datetime(final_result['Date']).dt.dayofweek
final_result['Week'] = pd.to_datetime(final_result['Date']).dt.isocalendar().week
final_result['Is_month_start'] = pd.to_datetime(final_result['Date']).dt.is_month_start
# #Start of month you get paid and can spend money on beers
final_result['Is_month_end'] = pd.to_datetime(final_result['Date']).dt.is_month_end
final_result['Is_year_start'] = pd.to_datetime(final_result['Date']).dt.is_year_start
final_result['Is_year_end'] = pd.to_datetime(final_result['Date']).dt.is_year_end
final_result['Is_weekend'] = np.where(final_result['Dayofweek'].isin([5,6]),1,0)
final_result['Is_weekday'] = np.where(final_result['Dayofweek'].isin([0,1,2,3,4]),1,0)

#SEASON
def get_season(date_time):
    # dummy leap year to include leap days(year-02-29) in our range
    leap_year = 2000
    seasons = [('winter', (date(leap_year, 1, 1), date(leap_year, 3, 20))),
               ('spring', (date(leap_year, 3, 21), date(leap_year, 6, 20))),
               ('summer', (date(leap_year, 6, 21), date(leap_year, 9, 22))),
               ('autumn', (date(leap_year, 9, 23), date(leap_year, 12, 20))),
               ('winter', (date(leap_year, 12, 21), date(leap_year, 12, 31)))]

    if isinstance(date_time, datetime):
        date_time = date_time.date()
    # we don't really care about the actual year so replace it with our dummy leap_year
    date_time = date_time.replace(year=leap_year)
    # return season our date falls in.
    return next(season for season, (start, end) in seasons
                if start <= date_time <= end)


# The apply method calls a function on each row
final_result['Season'] = pd.to_datetime(final_result['Date']).apply(get_season)

#BIRTHDAYS    
val = [1150200, 1137254, 1251302, 1177421, 1185114, 1242651, 1361300, 1347711, 1278152, 1312967, 1198601, 1257869]
#https://knoema.com/infographics/fjsoxbc/which-are-the-busiest-months-for-births

final_result['Birthdays'] = final_result['Month']
for x in range (1,13):
    final_result.loc[final_result['Month'] == x, 'Birthdays'] = val[x-1]/sum(val)
final_result

### IsPromo

In [ ]:
final_result['ts_promo'] = final_result['ts_id'].astype(str) + final_result['isPromo'].astype(str)
final_result['store_promo'] = final_result['Store'].astype(str) + final_result['isPromo'].astype(str)
final_result['segment_promo'] = final_result['Segment'].astype(str) + final_result['isPromo'].astype(str)
final_result['brand_promo'] = final_result['Brand'].astype(str) + final_result['isPromo'].astype(str)
final_result['sku_promo'] = final_result['SKU'].astype(str) + final_result['isPromo'].astype(str)

final_result['dom_promo'] = final_result['Day'].astype(str) + final_result['isPromo'].astype(str)
final_result['dow_promo'] = final_result['Dayofweek'].astype(str) + final_result['Dayofweek'].astype(str)

#### Holidays

In [ ]:

holidays2015 = pd.read_csv("data/holidays2015.csv")
holidays2016 = pd.read_csv("data/holidays2016.csv")
holidays2017 = pd.read_csv("data/holidays2017.csv")

hol_nam1 = pd.DataFrame(data=holidays2015["Name"].unique())

hol_nam2 = pd.DataFrame(data=holidays2016["Name"].unique())

hol_nam3 = pd.DataFrame(data=holidays2017["Name"].unique())

date_time1 = pd.to_datetime(holidays2015.pop('Date'), format='%d-%b')
holidays2015['Date'] = date_time1
holidays2015["Date"]=holidays2015["Date"].dt.strftime("2015-%m-%d")

date_time2 = pd.to_datetime(holidays2016.pop('Date'), format='%d-%b')
holidays2016['Date'] = date_time2
holidays2016["Date"]=holidays2016["Date"].dt.strftime("2016-%m-%d")

date_time3 = pd.to_datetime(holidays2017.pop('Date'), format='%d-%b')
holidays2017['Date'] = date_time3
holidays2017["Date"]=holidays2017["Date"].dt.strftime("2017-%m-%d")

result_holidays = pd.merge(
    hol_nam1,
    hol_nam2,
    how="outer"
)

result_holidays = pd.merge(
    result_holidays,
    hol_nam3,
    how="outer"
)
names_holidays = list(result_holidays[0])

matrix = np.zeros((len(final_result),len(names_holidays)))
holidays_df = pd.DataFrame(matrix,
                   columns=names_holidays)

final_result = final_result.join(holidays_df)

aux = 0
for index, row in final_result.iterrows():
    if row['Year']==2015:
        indexes = [i for i, e in enumerate(list(holidays2015['Date'])) if e == row['Date']]
        if indexes :
            for ind in indexes:
                final_result.at[index, holidays2015['Name'][ind]]=1
                final_result.at[index,"Holiday"] = holidays2015['Name'][ind]
        else:
            final_result.at[index, "Holiday"]="Not Holiday"

    if row['Year']==2016:
        indexes = [i for i, e in enumerate(list(holidays2016['Date'])) if e == row['Date']]
        if indexes :
            for ind in indexes:
                final_result.at[index,holidays2016['Name'][ind]]=1
                final_result.at[index, "Holiday"]=holidays2016['Name'][ind]
        else:
            final_result.at[index, "Holiday"]="Not Holiday"

    if row['Year']==2017:
        indexes = [i for i, e in enumerate(list(holidays2017['Date'])) if e == row['Date']]
        if indexes :
            for ind in indexes:
                final_result.at[index , holidays2017['Name'][ind]]=1
                final_result.at[index, "Holiday"]=holidays2017['Name'][ind]
        else:
            final_result.at[index, "Holiday"]="Not Holiday"


final_result

In [ ]:
final_result = final_result.drop(holidays_df,axis=1)
final_result

In [ ]:
#TEST SET
final_result_test['Year'] = pd.to_datetime(final_result_test['Date']).dt.year
final_result_test['Month'] = pd.to_datetime(final_result_test['Date']).dt.month
final_result_test['Day'] = pd.to_datetime(final_result_test['Date']).dt.day
final_result_test['Dayofweek'] = pd.to_datetime(final_result_test['Date']).dt.dayofweek
final_result_test['Week'] = pd.to_datetime(final_result_test['Date']).dt.isocalendar().week
final_result_test['Is_month_start'] = pd.to_datetime(final_result_test['Date']).dt.is_month_start
# #Start of month you get paid and can spend money on beers
final_result_test['Is_month_end'] = pd.to_datetime(final_result_test['Date']).dt.is_month_end
final_result_test['Is_year_start'] = pd.to_datetime(final_result_test['Date']).dt.is_year_start
final_result_test['Is_year_end'] = pd.to_datetime(final_result_test['Date']).dt.is_year_end
final_result_test['Is_weekend'] = np.where(final_result_test['Dayofweek'].isin([5,6]),1,0)
final_result_test['Is_weekday'] = np.where(final_result_test['Dayofweek'].isin([0,1,2,3,4]),1,0)

#SEASON
# The apply method calls a function on each row
final_result_test['Season'] = pd.to_datetime(final_result_test['Date']).apply(get_season)

#BIRTHDAYS    
val = [1150200, 1137254, 1251302, 1177421, 1185114, 1242651, 1361300, 1347711, 1278152, 1312967, 1198601, 1257869]
#https://knoema.com/infographics/fjsoxbc/which-are-the-busiest-months-for-births

final_result_test['Birthdays'] = final_result_test['Month']
for x in range (1,13):
    final_result_test.loc[final_result_test['Month'] == x, 'Birthdays'] = val[x-1]/sum(val)

### IsPromo

In [ ]:
final_result_test['ts_promo'] = final_result_test['ts_id'].astype(str) + final_result_test['isPromo'].astype(str)
final_result_test['store_promo'] = final_result_test['Store'].astype(str) + final_result_test['isPromo'].astype(str)
final_result_test['segment_promo'] = final_result_test['Segment'].astype(str) + final_result_test['isPromo'].astype(str)
final_result_test['brand_promo'] = final_result_test['Brand'].astype(str) + final_result_test['isPromo'].astype(str)
final_result_test['sku_promo'] = final_result_test['SKU'].astype(str) + final_result_test['isPromo'].astype(str)

final_result_test['dom_promo'] = final_result_test['Day'].astype(str) + final_result_test['isPromo'].astype(str)
final_result_test['dow_promo'] = final_result_test['Dayofweek'].astype(str) + final_result_test['isPromo'].astype(str)

#### Holidays test

In [ ]:
matrix = np.zeros((len(final_result_test),len(names_holidays)))
holidays_df = pd.DataFrame(matrix,
                   columns=names_holidays)

aux = 0
for index, row in final_result_test.iterrows():
    if row['Year']==2015:
        indexes = [i for i, e in enumerate(list(holidays2015['Date'])) if e == row['Date']]
        if indexes :
            for ind in indexes:
                final_result_test.at[index, holidays2015['Name'][ind]]=1
                final_result_test.at[index,"Holiday"] = holidays2015['Name'][ind]
        else:
            final_result_test.at[index, "Holiday"]="Not Holiday"

    if row['Year']==2016:
        indexes = [i for i, e in enumerate(list(holidays2016['Date'])) if e == row['Date']]
        if indexes :
            for ind in indexes:
                final_result_test.at[index,holidays2016['Name'][ind]]=1
                final_result_test.at[index, "Holiday"]=holidays2016['Name'][ind]
        else:
            final_result_test.at[index, "Holiday"]="Not Holiday"

    if row['Year']==2017:
        indexes = [i for i, e in enumerate(list(holidays2017['Date'])) if e == row['Date']]
        if indexes :
            for ind in indexes:
                final_result_test.at[index , holidays2017['Name'][ind]]=1
                final_result_test.at[index, "Holiday"]=holidays2017['Name'][ind]
        else:
            final_result_test.at[index, "Holiday"]="Not Holiday"

final_result_test

In [ ]:
final_result_test = final_result_test.drop(['Lailat al-Qadr','June Solstice','Eid al-Fitr','Day of Knowledge','Eid al-Adha','Muharram','September Equinox'],axis=1)
final_result_test

# Data cleaning

In [ ]:
#Find if there are duplicates
final_result = pd.DataFrame.drop_duplicates(final_result)# Nothing happens
beer_test = pd.DataFrame.drop_duplicates(beer_test)# Nothing happens


In [ ]:
#Find if there are nan
final_result.isnull().sum().sum() #No nan found
beer_test.isnull().sum().sum() #No nan found

## Save

In [ ]:
final_result.to_csv("train_preprocessed.csv", index=False)
final_result_test.to_csv("test_preprocessed.csv", index=False)

## Algorithm 1: avg (best)

In [ ]:
trainData = pd.read_csv("train_preprocessed.csv",dtype={'Store': object,'ts_promo': object,'store_promo': object,'dom_promo': object,'dow_promo': object})
trainData['Date'] = pd.to_datetime(trainData['Date'])

In [ ]:
testData = pd.read_csv("test_preprocessed.csv",dtype={'Store': object,'ts_promo': object,'store_promo': object,'dom_promo': object,'dow_promo': object})
testData['Date'] = pd.to_datetime(testData['Date'])

In [ ]:
for c in trainData.columns:
    col_type = trainData[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        trainData[c] = trainData[c].astype('category')

In [ ]:
for c in testData.columns:
    col_type = testData[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        testData[c] = testData[c].astype('category')

In [ ]:
print(trainData['Date'].is_monotonic_increasing)
print(testData['Date'].is_monotonic_increasing)
trainData = trainData.sort_values(by=['Date'])
testData = testData.sort_values(by=['Date'])

In [ ]:
X = trainData.drop(['Sales','Date'], axis=1)#Date
y = trainData['Sales']

In [ ]:
var_categ =[]
for c in X.columns:
    col_type = X[c].dtype
    if col_type.name == 'category':
        var_categ.append(c)

In [ ]:
lgbm = LGBMRegressor(
    boosting_type = 'dart',
    learning_rate = 0.35878911553612336, 
    max_depth = 1,
    metric='rmse',
    n_estimators = 524,
    num_leaves = 21,
    subsample_for_bin = 199000,
    colsample_bytree = 0.8,
    subsample = 1.0,
    silent=True
    )
testX = testData.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
lgbmfit = lgbm.fit(X,y)
lgbm_prediction = lgbm.predict(testX)

testData['Sales'] = lgbm_prediction
testData = testData.sort_values(by=['id'])
testData[['id','Sales']].to_csv("LGBMsubmission.csv", index=False)
lgbm_prediction = copy.copy(testData['Sales'])

In [ ]:
catboost = CatBoostRegressor(
    n_estimators=64,
    learning_rate=0.14038578250228806,
    depth=8,
    bagging_temperature=0.1075748346436656,
    border_count = 213,
    l2_leaf_reg = 12,
    random_strength = 5.03421359307061e-09,
    cat_features=var_categ,
    thread_count=-1,
    silent=True
)
testX = testData.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
catboost.fit(X,y)
catboost_prediction = catboost.predict(testX)
testData['Sales'] = catboost_prediction
testData = testData.sort_values(by=['id'])

In [ ]:
lgbm_prediction.to_numpy()
avg_prediction = np.mean([lgbm_prediction, catboost_prediction], axis=0)
testData['Sales'] = avg_prediction
testData = testData.sort_values(by=['id'])
testData[['id','Sales']].to_csv("avg1.csv", index=False)
testData[['id','Sales']]

# Algorithm 2: avg (other submission chosen)

In [ ]:
catboost = CatBoostRegressor(
    n_estimators=350,
    learning_rate=0.427912392471475789,
    depth=8,
    bagging_temperature=0.1080748346321565,
    border_count = 210,
    l2_leaf_reg = 10,
    random_strength = 5.03391369357891e-09,
    cat_features=var_categ,
    thread_count=-1,
    silent=True
)

In [ ]:
testX = testData.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
catboost.fit(X,y)
catboost_prediction2 = catboost.predict(testX)

testData['Sales'] = catboost_prediction2
testDataSorted = testData.sort_values(by=['id'])

In [ ]:
lgbm = LGBMRegressor(
    boosting_type = 'dart',
    learning_rate = 0.039691509923932582, 
    max_depth = 10,
    metric='rmse',
    n_estimators = 330,
    num_leaves = 21,
    subsample_for_bin = 201000,
    colsample_bytree = 0.8,
    subsample = 1.0,
    silent=True
    )

In [ ]:
testX = testData.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
lgbm.fit(X,y)
lgbm_prediction2 = lgbm.predict(testX)

testData['Sales'] = lgbm_prediction2
testData = testData.sort_values(by=['id'])

In [ ]:
lgbm_prediction

In [ ]:
catboost_prediction

In [ ]:
lgbm_prediction2

In [ ]:
catboost_prediction2

In [ ]:
avg_prediction = np.mean([lgbm_prediction, catboost_prediction,lgbm_prediction2, catboost_prediction2], axis=0)
testData['Sales'] = avg_prediction
testData = testData.sort_values(by=['id'])
testData[['id','Sales']].to_csv("avg2.csv", index=False)